In [ ]:
# Faktor modelering med maskin modelering
# Bygge Modell
# Mål - trene på programmering, maskinlæring og finansiell modellering

#Inspirasjon: https://medium.com/@yannisyuan/factor-investing-with-machine-learning-822fb62948c3

# git link: https://github.com/simbje/factor_modelling_machine.git

In [20]:
# Prerequisites

import pandas as pd
import numpy as np


In [21]:
#load file


#data = pd.read_csv('data.csv')
data = pd.read_csv(r'C:\Users\simbj\OneDrive\Python\data_factor\data\data_factor.csv')


In [18]:
# data eexploratin
data.dtypes

#

permno          int64
DATE            int64
mvel1         float64
beta          float64
betasq        float64
               ...   
retvol        float64
std_dolvol    float64
std_turn      float64
zerotrade     float64
sic2          float64
Length: 97, dtype: object

# Defining Fama-french model

start by selecting Fama-French-Carhart factors, namely market (“beta”), size (“mvel1”), value (“bm”) and momentum (“mom12m”), from the characteristics dataset

In [19]:
# Setting columns, choosing columns

cols = ['exret', 'beta', 'mvel1', 'bm', 'mom12m']  # Fama-French-Carhart factors
data_pivot = data[cols].unstack().swaplevel(axis=1).sort_index(axis=1).reindex(cols, axis=1, level=1)
data_pivot

KeyError: "['exret'] not in index"

We build a cross-sectional OLS estimator with statsmodels for pandas DataFrame in Python.

In [ ]:
import statsmodels.api as sm

def xs_ols(row_today):
    '''Build a cross-sectional OLS model and output the coefficient slopes at each timestamp'''
    
    df = row_today.unstack()
    df = df.dropna() # statesmodels OLS cannot handle missing values
    try:  # in case of error
        model = sm.OLS(df.iloc[:, 0], sm.add_constant(df.iloc[:, 1:])).fit()  
        if model.nobs > 2*(len(df.columns)-1):   # make sure n > 2k, n obs and k factors
            ans = np.r_[model.params.values, model.rsquared_adj, model.nobs] # output coeff and R2_adj & # of obs
        else:
            ans = np.empty((1, df.shape[1]+2)).flatten()
            ans[-1] = model.nobs
            ans[:-1] = np.nan
    except:                     # output empty array
        ans = np.empty((1, df.shape[1]+2)).flatten()
        ans[:-1] = np.nan
    return ans

df_ols = data_pivot.loc['1975':].progress_apply(xs_ols, axis=1)
df_ols = pd.DataFrame(np.stack(df_ols.values), index=df_ols.index, columns=['const']+cols[1:]+['Adj. R2', 'No. Obs'])
df_ols

In [ ]:
## Hello world, i am the git ###